In [1]:
import os
from openpyxl import Workbook
from openpyxl import load_workbook


''' 处理的最大行和列 '''
MAX_COLS = 25
MAX_ROWS = 2000


'''
根据行和列位置返回excel表格格式的位置。
行和列都是从0开始计数，最多支持25列数据
'''
def ws_loc(row, col):
    row = row + 1
    c = chr(ord('A') + col)
    loc = c + str(row)
    return loc


'''
Hash 数据合并
'''
def hash_merge(x, y):
    z = x.copy()
    for k,v in y.items():
        v = str(v).strip()
        if k in z:
            if not v == '':
                z[k] = v
        else:
            z[k] = v
    return z

'''
两级 Hash 数据合并
'''
def info_merge(x, y):
    z = x.copy()
    for k,v in y.items():
        if k in z:
            z[k] = hash_merge(z[k], v)
        else:
            z[k] = v
    return z


a = {'a':'2', 'b':'1'}
b = {'a':'3', 'b':'', 'c':'3'}
c = hash_merge(a, b)
print(c)
    

{'a': '3', 'b': '1', 'c': '3'}


In [2]:
'''
读 excel 表格并分析出每一标签页的数据
'''
def read_xls(path, index_key):
    print('\n============ Parse file : ', path)
    wb = load_workbook(path)
    
    info = {}
    for sheet in wb:
        data = {}
        header = {}
        found = False
        index = 0
        
        print('Parse Worksheet : ', sheet.title, sheet.max_column, sheet.max_row)
        
        # 遍历标题栏，查找索引：姓名
        cols = sheet.max_column if sheet.max_column < MAX_COLS else MAX_COLS
        for i in range(cols):
            loc = ws_loc(0, i)
            if sheet[loc].value is None:
                break
            value = str(sheet[loc].value)
            value = ''.join(value.split()).strip()
            print(loc, value)
            header[i] = value
            if index_key == value:
                found = True
                index = i
        
        # 未发现索引，忽略改表格
        if not found:
            break

        # 遍历表格，生成 hash 数据
        rows = sheet.max_row if sheet.max_row < MAX_ROWS else MAX_ROWS
        for r in range(rows):
            loc = ws_loc(r, index)
            oui = sheet[loc].value
            oui = str(oui).strip()

            # 没有索引内容，则不处理
            if oui is None or oui == index_key:
                continue

            # 一行记录生成
            record = {}
            for c in range(cols):
                if not c in header:
                    continue
                loc = ws_loc(r, c)
                val = '' if sheet[loc].value is None else sheet[loc].value
                record[header[c]] = str(val).strip()

            # 更新 hash 表
            if oui in data:
                print("record repeat : ", oui)
            data[oui] = record
            
        # 合并各个表格的数据
        info = info_merge(info, data)
    # 返回 excel 最终数据
    return info



'''
遍历指定目录下的 execl 表格并生成 hash 数据（默认以mac作为唯一标识）
仅支持 .xlsx 后缀
'''
def process_excel_dir(dpath, key = 'mac'):
    all_info = {}
    for root, dlist, flist in os.walk(dpath):
        flist.sort()
        for f in flist:
            fpath = os.path.join(root, f)
        
            # 只支持的 excel 表格类型
            if not f.endswith('.xlsx'):
                continue
            
            # 临时文件及隐藏文件不遍历
            if f.startswith('.'):
                continue
        
            # 读取并合并 excel 文件数据
            info = read_xls(fpath, key)
            all_info = info_merge(all_info, info)
    return all_info




In [3]:


header = ['mac', 'status', 'sn', 'sip']

def print_record(info):
    for k,v in info.items():
        record = ''
        # print(v)
        for i in header:
            if i in v :
                record = record + v[i] + ','
            else:
                record += ','
        print(record)

        
#all_info = process_excel_dir('info')
dev_info = read_xls('xlsx/dev-list.xlsx', 'mac')
sip_info = read_xls('xlsx/sip-list.xlsx', 'mac')
acc_info = read_xls('xlsx/acc-list.xlsx', 'sip')

dev_info = dict((k.lower(), v) for k,v in dev_info.items())
sip_info = dict((k.lower(), v) for k,v in sip_info.items())
acc_info = dict((k.lower(), v) for k,v in acc_info.items())

print('dev_info = ', len(dev_info))
print('sip_info = ', len(sip_info))
print('acc_info = ', len(acc_info))

#idx = '868753037441@ims.ge.chinamobile.com'
#print_record(acc_info)
#print(acc_info)
#print('sip:', acc_info[idx])

out_info = {}
for mac, dev in sip_info.items():
    if mac in dev_info:
        sn1 = dev_info[mac]['sn'].lower().strip()
        sn2 = dev['sn'].lower().strip()
        sip = dev['sip'].strip()
        if sip in acc_info:
            dev['account'] = acc_info[sip]['account']
            dev['passwd'] = acc_info[sip]['passwd']
        
        if sn1 == sn2:
            out_info[mac] = dev
        else:
            print("SN Warning : mac=", mac, "sn1=", sn1, "sn2=", sn2)
    else:
        print('not reset device :', mac)
    
print('acc_info = ', len(out_info))






============ Parse file :  xlsx/dev-list.xlsx
Parse Worksheet :  Sheet1 43 1553
A1 id
B1 pid
C1 lastUpdate
D1 status
E1 info
F1 historyInfo
G1 执行时间
H1 completeTime
I1 badId
J1 data
K1 accessqueue
L1 deviceId
M1 id
N1 sn
O1 mac
P1 lanIp
Q1 wanIp
R1 hardwareId
S1 hardware
T1 softwareId
U1 softwareVersion
V1 extSoftWareVersion
W1 remark
X1 devDesp
Y1 devOem
record repeat :  0C:11:05:06:40:56
record repeat :  AA:AA:AA:AA:AA:AA
record repeat :  AA:AA:AA:AA:AA:AA
record repeat :  0C:11:05:06:40:B7

============ Parse file :  xlsx/sip-list.xlsx
Parse Worksheet :  查找 17 251
A1 注册状态
B1 在线状态
C1 终端会场名称
D1 终端备注
E1 系统管理域
F1 sip
G1 终端类型
H1 网元IP
I1 网元真实IP
J1 软件版本
K1 所属分组
L1 最近上线时间
M1 创建时间
N1 mac
O1 sn
P1 客户名称
Q1 查找
record repeat :  0c:11:05:06:32:22
record repeat :  20:93:4d:34:a0:1e
record repeat :  20:93:4d:80:f1:ac
record repeat :  20:93:4d:77:cc:4b

============ Parse file :  xlsx/acc-list.xlsx
Parse Worksheet :  Sheet1 21 323
A1 人员
B1 配置完成情况（是/否/离线）
C1 Source.Name
D1 成员号码
E1 列1
F1 成员归属省
G1 订单状态

In [4]:
def format_cell(cell):
    cell.number_format = 'General'

def format_ws(ws):
    ws.column_dimensions['A'].width = 22.0
    ws.column_dimensions['B'].width = 30.0
    ws.column_dimensions['C'].width = 40.0
    ws.column_dimensions['D'].width = 40.0
    ws.column_dimensions['E'].width = 40.0
    
def hash2excel(data, file, header):
    wb = Workbook()
    ws = wb.create_sheet('info', 0)
    format_ws(ws)
    head_count = len(header)
    row = 0
    for c in range(head_count):
        loc = ws_loc(0, c)
        ws[loc] = header[c]

    for k, v in data.items():
        row = row + 1
        for c in range(head_count):
            loc = ws_loc(row, c)
            key = header[c]
            val = '' if not key in v else v[key]
            format_cell(ws[loc])
            ws[loc] = val
            # print(loc, val)

    wb.save(file)

out_header = ['mac', 'sn', 'sip', 'account', 'passwd']
hash2excel(out_info, 'xlsx/output.xlsx', out_header)
